In [44]:
# Script to prepare the data from CS3 of climate indicators proyections to run Calliope
# Only UK and IE are included
# Noelia Otero. Energy-Climate Hacktahon 2021

import pandas as pd
import geopandas as gpd
# Operations on geometries
import shapely
import numpy as np
import math
import xarray as xr
from datetime import datetime

In [45]:
# Energy infodata
path_clims = '/home/users/train028/capacity_climproj/'
pv_rcp26 = pd.read_csv(path_clims + 'P_CMI5_ReMO_CM20_SPV_NA---_Euro_NUT0_S197101010130_E209912312230_CFR_TIM_03h_NA-_cdf_org_01_RCP26_NA---_PhM02.csv',skiprows=range(0, 51))
pv_rcp85 = pd.read_csv(path_clims + 'P_CMI5_ReMO_CM20_SPV_NA---_Euro_NUT0_S197101010130_E209912312230_CFR_TIM_03h_NA-_cdf_org_01_RCP85_NA---_PhM02.csv',skiprows=range(0, 51))
wind_rcp26 = pd.read_csv(path_clims + 'P_CMI5_ReMO_CM20_WON_0100m_Euro_NUT0_S197101010000_E209912312100_CFR_TIM_03h_NA-_cdf_org_01_RCP26_NA---_PhM01.csv',skiprows=range(0, 51))
wind_rcp85 = pd.read_csv(path_clims + 'P_CMI5_ReMO_CM20_WON_0100m_Euro_NUT0_S197101010000_E209912312100_CFR_TIM_03h_NA-_cdf_org_01_RCP85_NA---_PhM01.csv', skiprows=range(0, 51))

#HRE_rcp26 = pd.read_csv(path_clims + 'P_CMI5_ReMO_CM20_HRE_NA---_Euro_NUT0_S197101010000_E209912312300_CFR_TIM_01d_NA-_noc_org_NA_RCP26_NA---_StRnF.csv',skiprows=range(0, 61))
#HRE_rcp85 = pd.read_csv(path_clims + 'P_CMI5_ReMO_CM20_HRE_NA---_Euro_NUT0_S197101010000_E209912312300_CFR_TIM_01d_NA-_noc_org_NA_RCP85_NA---_StRnF.csv',skiprows=range(0, 61))
#HROR_rcp26 = pd.read_csv(path_clims + 'P_CMI5_ReMO_CM20_HRO_NA---_Euro_NUT0_S197101010000_E209912312300_CFR_TIM_01d_NA-_noc_org_NA_RCP26_NA---_StRnF.csv',skiprows=range(0, 61))
#HROR_rcp85 = pd.read_csv(path_clims + 'P_CMI5_ReMO_CM20_HRO_NA---_Euro_NUT0_S197101010000_E209912312300_CFR_TIM_01d_NA-_noc_org_NA_RCP85_NA---_StRnF.csv',skiprows=range(0, 61))

wind_OF_rcp26 = pd.read_csv(path_clims + 'P_CMI5_ReMO_CM20_WOF_0100m_Euro_MAR0_S197101010000_E209912312100_CFR_TIM_03h_NA-_cdf_org_01_RCP26_NA---_PhM01.csv',skiprows=range(0, 51))
wind_OF_rcp85 = pd.read_csv(path_clims + 'P_CMI5_ReMO_CM20_WOF_0100m_Euro_MAR0_S197101010000_E209912312100_CFR_TIM_03h_NA-_cdf_org_01_RCP85_NA---_PhM01.csv', skiprows=range(0, 51))
# read daily
path_clims_day ='/home/users/train028/daily_CF/'
# read Demand RCP
demand_rcp26 = pd.read_csv(path_clims_day + 'P_CMI5_ReMO_CM20_EDM_0000m_Euro_NUT0_S197101010000_E209912310000_PWR_TIM_01d_NA-_noc_org_01_RCP26_NA---_GamNT.csv',skiprows=range(0, 53))
demand_rcp85 = pd.read_csv(path_clims_day + 'P_CMI5_ReMO_CM20_EDM_0000m_Euro_NUT0_S197101010000_E209912310000_PWR_TIM_01d_NA-_noc_org_01_RCP85_NA---_GamNT.csv',skiprows=range(0, 53))

In [49]:
def savedata(mdat,selcol, fname,data,daily):
    # select IE & UK 
    selcol.append('Date')
    mdat_nat=mdat[selcol]
    #mdat_nat.rename(columns = {'Date':'time','UK':'GBR'}, inplace=True)
    mdat_nat.columns= ['IRL','GBR','time']
    mdat_nat = mdat_nat[['time','IRL','GBR']]
    if daily:
        mdat_nat['time'] =  pd.to_datetime(mdat_nat['time'], format='%Y-%m-%d %H:%M:%S')
        day_dat = mdat_nat.resample('d', on='time').mean()
        day_dat.reset_index(level=0, inplace=True)
        day_dat.to_csv(fname + '_daily' + ".csv", sep = ",", index = False)
        
        
    if (data=="demand"):
        mdat_nat['time'] =  pd.to_datetime(mdat_nat['time'], format='%Y-%m-%d %H:%M:%S')
        mdat_nat[['IRL','GBR']]=(mdat_nat[['IRL','GBR']].values*(-1)/24)*0.001 # need to convert to daily mean power 
        
        #mdat_nat.reset_index(level=0, inplace=True)
    mdat_nat.to_csv(fname + ".csv", sep = ",", index = False)
    return(mdat_nat)
    

In [260]:
savedata(pv_rcp26,['IE','UK'], "pv_day_rcp26", daily= 'TRUE')
savedata(pv_rcp85,['IE','UK'], "pv_day_rcp85", daily= 'TRUE')


,time,IRL,GBR
0,1971-01-01 01:30:00,0.000000,0.000000
1,1971-01-01 04:30:00,0.000000,0.000000
2,1971-01-01 07:30:00,0.000161,0.003399
3,1971-01-01 10:30:00,0.222290,0.241390
4,1971-01-01 13:30:00,0.353952,0.296971
5,1971-01-01 16:30:00,0.046895,0.018430
6,1971-01-01 19:30:00,0.000000,0.000000
7,1971-01-01 22:30:00,0.000000,0.000000
8,1971-01-02 01:30:00,0.000000,0.000000
9,1971-01-02 04:30:00,0.000000,0.000000
